# SVM Classifier without PCA

# SVM Classifier with PCA

In [1]:
# general imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, log_loss

In [2]:
# loading in original data sets
df_training = pd.read_csv('../data/archive/train.csv')
df_testing = pd.read_csv('../data/archive/test.csv')

# loading in pca data sets
df_training_pca = pd.read_csv('../data/archive/train_pca.csv')
df_testing_pca = pd.read_csv('../data/archive/test_pca.csv')

In [3]:
#Separating X and y training and testing
x_train = df_training.iloc[:, :-1]
y_train = df_training.iloc[:,-1]

x_test = df_testing.iloc[:, :-2]
y_test = df_testing.iloc[:,-1]

# get the pca training and test data

x_train_pca = df_training_pca.iloc[:, :-1]
y_train_pca = df_training_pca.iloc[:,-1]

x_test_pca = df_testing_pca.iloc[:, :-1]
y_test_pca = df_testing_pca.iloc[:,-1]

In [ ]:
#Initializing SVC

svm = SVC()

In [ ]:
#Making a grid of values we want our grid search to test to find the best parameters
grid_values = {'kernel': ['linear', 'rbf'], 'C': [1,10,100,1000], 'decision_function_shape': ['ovr', 'ovo']}

svc_classifier = GridSearchCV(estimator = svm, param_grid = grid_values, cv = StratifiedKFold(n_splits = 5),
                      scoring = ['accuracy', 'f1_micro'], refit = False, verbose = 0)

svc_model = svc_classifier.fit(x_train, y_train)

In [ ]:
#initializing our lists so we can later store the best params that achieve best scores across metrics
accuracy_best_params = []
f1_best_params = []
precision_best_params = []

accuracy_best_params.append(svc_model.cv_results_['params'][ np.argmin(svc_model.cv_results_['rank_test_accuracy'])])
f1_best_params.append(svc_model.cv_results_['params'][ np.argmin(svc_model.cv_results_['rank_test_f1_micro'])])
precision_best_params.append(svc_model.cv_results_['params'][ np.argmin(svc_model.cv_results_['rank_test_precision_micro'])])

In [ ]:
accuracy_best_params

In [ ]:
f1_best_params

In [ ]:
precision_best_params

In [ ]:
accuracy_test_score = []

svc_clf = SVC(kernel = accuracy_best_params[0].get('kernel'), C = accuracy_best_params[0].get('C'),
              decision_function_shape = accuracy_best_params[0].get('decision_function_shape'))
model = svc_clf.fit(x_train, y_train)

y_pred = model.predict(x_test)

accuracy_test_score.append(accuracy_score(y_test, y_pred))

In [ ]:
accuracy_test_score

In [ ]:
f1_test_score = []

svc_clf = SVC(kernel = f1_best_params[0].get('kernel'), C = f1_best_params[0].get('C'),
              decision_function_shape = f1_best_params[0].get('decision_function_shape'))
model = svc_clf.fit(x_train, y_train)

y_pred = model.predict(x_test)

accuracy_test_score.append(f1_score(y_test, y_pred, average = 'micro'))

In [ ]:
f1_test_score

In [ ]:
precision_test_score = []

svc_clf = SVC(kernel = f1_best_params[0].get('kernel'), C = f1_best_params[0].get('C'),
              decision_function_shape = f1_best_params[0].get('decision_function_shape'))
model = svc_clf.fit(x_train, y_train)

y_pred = model.predict(x_test)

precision_test_score.append(precision_score(y_test, y_pred, average = 'micro'))

In [ ]:
precision_test_score